In [1]:
#%pip install python-dotenv

In [30]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Access your API key
api_key = os.getenv("GEMINI_API_KEY")
from google import genai
client = genai.Client(api_key=api_key)

In [85]:
import requests, re

def fetch_passage_n_tracker(input_tracker):
    """
    Fetches a passage from Meditations by Marcus Aurelius from Project Gutenberg.

    Args:
        chapter_name: The name of the chapter (e.g., "THE FIRST BOOK").
        para_name: The paragraph number within the chapter (e.g., "II.").

    Returns:
        The text of the passage, or None if the passage is not found.
        Also returns the next passage tracker i.e. {chapter_name-para_name} to be read next time.
    """
    url = 'https://www.gutenberg.org/cache/epub/2680/pg2680.txt'
    response = requests.get(url)
    response.raise_for_status()  # Raise an exception for bad status codes

    chapter_name = input_tracker.split('-')[0]
    para_name = input_tracker.split('-')[1]
    text = response.text
    # Find the start of the chapter
    chapter_start = text.find(chapter_name)
    if chapter_start == -1:
        return None  # Chapter not found

    # Find the start of the paragraph within the chapter
    para_start = text.find(para_name + " ", chapter_start) #Added space to avoid cases where paragraph name are substrings of other paragraphs
    if para_start == -1:
        para_start = text.find(para_name + "\n", chapter_start) #checking for newline character if the space search fails
        if para_start == -1:
            return None  # Paragraph not found
    
    match = re.search(r"\n\s*\n", text[para_start:])
    if match:
        para_end = para_start + match.start()
    else:
        para_end = len(text)  # No blank line found, go to end of text
    
    passage_text = text[para_start:para_end].strip()
    
    
    # Find next paragraph number (Roman numeral)
    next_para_match = re.search(r"\n([IVXLCDM]+)\.", text[para_end:])

    # Find next chapter title (uppercase words only)
    next_chapter_match = re.search(r"\n(THE [A-Z]+ BOOK)", text[para_end:])

    # Determine which comes first
    next_para_name = None
    next_chapter_name = chapter_name  # Default: same chapter
    
    if next_para_match:
        next_para_start = para_end + next_para_match.start()
        next_para_name = next_para_match.group(1) + "."

        # If a chapter title is found, check if it appears first
        if next_chapter_match:
            next_chapter_start = para_end + next_chapter_match.start()
            if next_chapter_start < next_para_start:
                next_chapter_name = next_chapter_match.group(1).strip()

    tracker = f"{next_chapter_name}-{next_para_name}" if next_para_name else 'THE FIRST BOOK-I.'
    
    return passage_text, tracker



# Query
# chapter_name = "THE FIRST BOOK"
# para_name = "XVII."
# passage, tracker = fetch_passage(chapter_name, para_name)

# tracker

In [97]:
def give_modern_interpretation(string):
    """
    Gives modern interpretation of the text passed.

    Args:
        passage: string

    Returns:
        String
    """
    
    system_prompt = """ You are a stoic philosopher and you will provide modern interpretation of the 
    following text (2-3 sentences). 
    """
    current_query = string
    
    prompt = f"{system_prompt}\n\nQuery: {current_query}"
    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=prompt
    )
    
    response_text = response.text.strip()
    return response_text

In [100]:
def give_daily_stoic_exercise(string):
    """
    Gives daily exercise

    Args:
        passage: string

    Returns:
        String
    """
    
    system_prompt = """ You are a stoic philosopher and you will suggest a practical daily exercise 
    based on this text (1 sentence).  
    """
    
    current_query = string
    
    prompt = f"{system_prompt}\n\nQuery: {current_query}"
    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=prompt
    )
    
    response_text = response.text.strip()
    return response_text 

In [103]:
def function_caller(func_name, params):
    """Simple function caller that maps function names to actual functions"""
    function_map = {
        "fetch_passage_n_tracker": fetch_passage_n_tracker,
        "give_modern_interpretation": give_modern_interpretation,
        "give_daily_stoic_exercise": give_daily_stoic_exercise
    }
    
    if func_name in function_map:
        return function_map[func_name](params)
    else:
        return f"Function {func_name} not found"

In [104]:
max_iterations = 3
last_response = None
iteration = 0
iteration_response = []

system_prompt = """ You are a book reader who is well versed with reading books and can read the book in a sequential order. 
You know how to break the book into passages and will break them up paragraph wise in order to facilitate daily reading.
We want to read the book Meditations by Marcus Aurelius. The book is organised into chapters which are named 
THE FIRST BOOK, THE SECOND BOOK and so on. The chapters are divided into paragraphs which are numbered in 
roman numerals i.e I., II., and so on. Your job is to fetch the passage corresponding to the chapter name and paragraph name 
for the user. You will also fetch the next chapter name and paragraph number, so we can keep track of up to where the user
have read the book. You will also provide a modern interpretation of the passage that is fetched. And lastly you will
provide a daily stoic exercise of the passage.

Respond with EXACTLY ONE of these formats:

1. FUNCTION_CALL: python_function_name|input

where python_function_name is one of the followin:
1. fetch_passage_n_tracker(string) Fetches a passage from Meditations by Marcus Aurelius from Project Gutenberg.

    Args:
        takes in input string in the format {chapter_name-para_name} where
        chapter_name: The name of the chapter (e.g., "THE FIRST BOOK").
        para_name: The paragraph number within the chapter (e.g., "II.").

    Returns:
        The text of the passage, or None if the passage is not found.
        Also returns the next passage tracker in the format {chapter_name-para_name} to be read next time.
        
2. give_modern_interpretation(string) Gives modern interpretation of the text of the passage. It takes text and returns text. 
3. give_daily_stoic_exercise(string)  Suggest a practical daily exercise based on the text of the passage. This takes a text 
and returns a text.

DO NOT include multiple responses. Give ONE response at a time.
"""

query= """Fetch the passage text corresponding to THE FIRST BOOK and paragraph II.."""

while iteration < max_iterations:
    
    print(f"\n--- Iteration {iteration + 1} ---")
    
    if last_response == None:
        current_query = query
    else:
        current_query = current_query + "\n\n" + " ".join(iteration_response)
        current_query = current_query + "  What should I do next?"
        
        
    # Get model's response
    prompt = f"{system_prompt}\n\nQuery: {current_query}"
    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=prompt
    )
    
    response_text = response.text.strip()
    print(f"LLM Response: {response_text}")
    
    if response_text.startswith("FUNCTION_CALL:"):
        response_text = response.text.strip()
        _, function_info = response_text.split(":", 1)
        func_name, params = [x.strip() for x in function_info.split("|", 1)]
        iteration_result = function_caller(func_name, params)
        
        
    print(f"  Result: {iteration_result}")
    last_response = iteration_result
    iteration_response.append(f"In the {iteration + 1} iteration you called {func_name} with {params} parameters, and the function returned {iteration_result}.")

    iteration = iteration + 1
    


--- Iteration 1 ---
LLM Response: FUNCTION_CALL: fetch_passage_n_tracker|THE FIRST BOOK-II.
  Result: ('II. Of him that brought me up, not to be fondly addicted to either of\r\nthe two great factions of the coursers in the circus, called Prasini,\r\nand Veneti: nor in the amphitheatre partially to favour any of the\r\ngladiators, or fencers, as either the Parmularii, or the Secutores.\r\nMoreover, to endure labour; nor to need many things; when I have\r\nanything to do, to do it myself rather than by others; not to meddle\r\nwith many businesses; and not easily to admit of any slander.', 'THE FIRST BOOK-III.')

--- Iteration 2 ---
LLM Response: FUNCTION_CALL: give_modern_interpretation|II. Of him that brought me up, not to be fondly addicted to either of\r\nthe two great factions of the coursers in the circus, called Prasini,\r\nand Veneti: nor in the amphitheatre partially to favour any of the\r\ngladiators, or fencers, as either the Parmularii, or the Secutores.\r\nMoreover, to endu